In [1]:
#!pip install matplotlib
#!pip install koeda
#!pip install transformers

In [2]:
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm
from transformers import AutoTokenizer

In [3]:
train = pd.read_csv("train.csv")
train

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0
...,...,...,...,...,...,...
9319,boostcamp-sts-v1-train-9319,petition-sampled,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요,0.2,0.0
9320,boostcamp-sts-v1-train-9320,petition-sampled,여성가족부의 폐지를 원합니드,여성가족부 폐지를 청원 합니다.,4.2,1.0
9321,boostcamp-sts-v1-train-9321,petition-sampled,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요,0.6,0.0
9322,boostcamp-sts-v1-train-9322,slack-sampled,오늘 못한 점심은 다음에 다시 츄라이 하기로 해요!!,오늘 못먹은 밥은 꼭 담에 먹기로 하고요!!,3.2,1.0


A행의 sen_2와 B행의 sen_1이 같은 경우를 구하자
A행의 sen_1, B행의 sen_2는 두 행의 label 중 최솟값

In [4]:
a = {'A' : [111, 222, 333, 444, 555, 666], 'B' : ['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff']}
df = pd.DataFrame(a)
print(df)

i = 0
df = df.append(df.loc[i], ignore_index = True)
print(df)
print(len(df))
df['A'].iloc[len(df)-1]= df['A'].iloc[1]
print(df)

     A    B
0  111  aaa
1  222  bbb
2  333  ccc
3  444  ddd
4  555  eee
5  666  fff
     A    B
0  111  aaa
1  222  bbb
2  333  ccc
3  444  ddd
4  555  eee
5  666  fff
6  111  aaa
7
     A    B
0  111  aaa
1  222  bbb
2  333  ccc
3  444  ddd
4  555  eee
5  666  fff
6  222  aaa


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
sen1_arr = []
sen2_arr = []

for i in range(0, 9324):
    sen1_arr.append(train.iloc[i][2])
    sen2_arr.append(train.iloc[i][3])

idx = []
for i in range(0, 9324):
    for j in range(0, 9324):
        if sen2_arr[i] == sen1_arr[j]:
            print(train['sentence_1'].iloc[j], ' |',train['sentence_2'].iloc[j], train['label'].iloc[j],' \n ', 
                  train['sentence_1'].iloc[i], ' | ',train['sentence_2'].iloc[i], train['label'].iloc[i], ' \n ')
            train = train.append(train.loc[i], ignore_index = True)
            idx = len(train)-1
            train['sentence_1'].iloc[idx] = train['sentence_1'].iloc[i]
            train['sentence_2'].iloc[idx] = train['sentence_2'].iloc[j]
            train['label'].iloc[idx]  = min(train['label'].iloc[i], train['label'].iloc[j])
            '''
            print(train[-1][2])
            temp[2] = train.iloc[i][2]
            temp[3] = train.iloc[j][3]
            temp[4] = min(train.iloc[i][4], train.iloc[j][4])
            train = pd.concat([train, temp])
            '''
            #idx.append([i,j])
#print(idx)

모두 해피 주말 ~!  | 모두 즐거운 주말 되세요! 4.0  
  모두 해삐 주말입니다!!!!  |  모두 해피 주말 ~! 4.2  
 


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


그만 좀 울고 동생연기 좀 제대로 시키세요  | 울지 말고 형이 제대로 행동하게 하세요. 1.8  
  영화마다 진지하게 일장연설하는 부분은 제발 이제 그만~  |  그만 좀 울고 동생연기 좀 제대로 시키세요 0.0  
 
이런게 진짜 영화다,  | 이건 진짜 영화다 3.4  
  이런게 바로 영화다.  |  이런게 진짜 영화다, 4.0  
 
김성태 원내대표를 폭행한 폭행범의 강력처벌을 요구합니다  | 원내대표 김성태 폭행한 가해자 강력처벌 촉구 4.0  
  김성태 원내대표 푀행 가해자에 대한 올바른 법집행 청원  |  김성태 원내대표를 폭행한 폭행범의 강력처벌을 요구합니다 3.2  
 
가상화폐 거래소 폐지를 반대합니다  | 암호화폐 거래소 폐지 반대 3.8  
  거래소 폐지를 금해주세요  |  가상화폐 거래소 폐지를 반대합니다 3.6  
 
나 또한 원주율을 위해..  | 나도... 0.2  
  원작에 대한 예우 차원에서 저 점수라도 주는거임  |  나 또한 원주율을 위해.. 0.0  
 
수능 연기를 부탁드립니다  | 입시를 연기해주세요 1.0  
  수능시험시간 연기해주세요  |  수능 연기를 부탁드립니다 2.8  
 
아 헐! 찾아보니 그렇네요!!  | 오 헐! 찾았어요!! 0.6  
  슬랙 내용 찾아보니 대략 6개월 전에 요청하셨던 것으로 추정되네요..!  |  아 헐! 찾아보니 그렇네요!! 0.8  
 
너무 분위기 좋았고 반가웠어요!!  | 분위기도 너무 좋고 즐거웠어요!! 3.0  
  대전 분이신줄은 몰랐는데 반가웠습니다! ㅋㅋ  |  너무 분위기 좋았고 반가웠어요!! 1.6  
 
생명의 위협을 받았던 걸까?  | 목숨이 위협받았습니까? 3.4  
  아니면, 내가 악몽을 꾸고 있는 걸까?  |  생명의 위협을 받았던 걸까? 0.0  
 
군사 쿠데타를 절대로 막아서는 안 됩니다  | 군사 쿠데타를 멈추지 마라 3.8  
  [가상화폐] 민주당은 영원히 재집권할 수 없게 됩니다  |  군사 쿠데타를 절대로 막아서는 안 됩니다 0.0

In [6]:
train.tail(10)

,id,source,sentence_1,sentence_2,label,binary-label
9325,boostcamp-sts-v1-train-1080,nsmc-sampled,영화마다 진지하게 일장연설하는 부분은 제발 이제 그만~,울지 말고 형이 제대로 행동하게 하세요.,0.0,0.0
9326,boostcamp-sts-v1-train-2890,nsmc-sampled,이런게 바로 영화다.,이건 진짜 영화다,3.4,1.0
9327,boostcamp-sts-v1-train-3257,petition-sampled,김성태 원내대표 푀행 가해자에 대한 올바른 법집행 청원,원내대표 김성태 폭행한 가해자 강력처벌 촉구,3.2,1.0
9328,boostcamp-sts-v1-train-4527,petition-sampled,거래소 폐지를 금해주세요,암호화폐 거래소 폐지 반대,3.6,1.0
9329,boostcamp-sts-v1-train-5402,nsmc-sampled,원작에 대한 예우 차원에서 저 점수라도 주는거임,나도...,0.0,0.0
9330,boostcamp-sts-v1-train-6479,petition-sampled,수능시험시간 연기해주세요,입시를 연기해주세요,1.0,1.0
9331,boostcamp-sts-v1-train-7270,slack-sampled,슬랙 내용 찾아보니 대략 6개월 전에 요청하셨던 것으로 추정되네요..!,오 헐! 찾았어요!!,0.6,0.0
9332,boostcamp-sts-v1-train-7572,slack-sampled,대전 분이신줄은 몰랐는데 반가웠습니다! ㅋㅋ,분위기도 너무 좋고 즐거웠어요!!,1.6,0.0
9333,boostcamp-sts-v1-train-7623,nsmc-sampled,"아니면, 내가 악몽을 꾸고 있는 걸까?",목숨이 위협받았습니까?,0.0,0.0
9334,boostcamp-sts-v1-train-8616,petition-sampled,[가상화폐] 민주당은 영원히 재집권할 수 없게 됩니다,군사 쿠데타를 멈추지 마라,0.0,0.0


NLP에서 Text Augmentation 방법은 크게 텍스트의 일부를 변형하여 데이터를 증강하는 방법과 생성모델을 사용하여 새로운 텍스트를 생성하여 데이터를 증강하는 방법이 있다. 
그 중에서 가장 손쉽게 접근할 수 있는 방법은 KoEDA 라이브러리를 사용하는 것이었다.
KoEDA는 EDA와 AEDA 논문에서 소개된 방식을 한국어 Wordnet 으로 Porting하여 공개한 오픈소스 라이브러리이다.

<EDA의 네 가지 기법> 

Easy Data Augmentation라는 논문에서는 데이터를 다음의 네 가지 기법을 통해 자연어 데이터를 증강하고자 한다.

* 유의어로 교체(Synonym Replacement, SR): 문장에서 랜덤으로 stop words가 아닌 n 개의 단어들을 선택해 임의로 선택한 동의어들 중 하나로 바꾸는 기법.

* 랜덤 삽입(Random Insertion, RI): 문장 내에서 stop word를 제외한 나머지 단어들 중에서, 랜덤으로 선택한 단어의 동의어를 임의로 정한다. 그리고 동의어를 문장 내 임의의 자리에 넣는걸 n번 반복한다.

* 랜덤 교체(Random Swap, RS): 무작위로 문장 내에서 두 단어를 선택하고 위치를 바꾼다. 이것도 n번 반복

* 랜덤 삭제(Random Deletion, RD): 확률 p를 통해 문장 내에 있는 각 단어들을 랜덤하게 삭제한다.

<EDA의 성능>

데이터셋이 적다는 가정에서, 데이터셋이 500개일 때 EDA를 포함하면 평균적으로 3%의 성능이 증가함을 확인할 수 있다. full set일 때도 EDA를 사용하면 평균적으로 성능의 향상이 있었다. 

하지만 우리가 사용할 BERT 등의 선학습 모델은 거대 데이터셋으로 선학습되었기에 데이터셋의 개선 효과를 못 볼 수도 있다고 한다.

<EDA 활용 팁>

한 문장에 대해서 몇 개의 문장을 만들건지에 따라 α값에 조정이 필요하며, 
4문장 이하는 p=0.1, 4문장 초과는 p=0.05 정도의 확률값으로 데이터를 변형하는게 가장 성능이 좋았다고 저술되어있다. 

하지만 텍스트 데이터의 특성상, 위치를 바꾸거나 일부 단어를 제거하는 것은 결국 본 문장의 의미를 손실시키는 행위이기 때문에 AEDA 방법론이 등장하게 된다.

<AEDA란>
AEDA는 문장을 손실시키지 않게 하기 위해 Special character를 문장 곳곳에 배치하는 방법론으로, 
역시 많은 특수문자가 들어가게 되면 성능이 떨어지기 때문에 적절한 확률값을 찾는 것이 중요하다.

***한글 자연어 처리 패키지 konlpy

konlpy는 형태소 등을 알아서 분석해주는 편리한 패키지이지만, konlpy 내 클래스는 Java 기반이기 때문에 그냥 pip install konlpy로 설치할 수 없다. 
아래와 같은 과정을 거쳐야 하는데

1. JAVA 설치
   https://www.oracle.com/java/technologies/javase-downloads.html
2. JAVA_HOME 환경변수 설정
3. JPype 다운로드 및 설치
   https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype
   파이썬 버전과 맞게 다운로드해야지 cmd창에서 'not a valid wheel filename' 에러가 안 뜬다
4. konlpy 설치

참고: https://byeon-sg.tistory.com/entry/자연어-처리-konlpy-설치-오류-okt에러-already-loaded-in-another-classloader-SystemErro-1 [wave:티스토리]

In [7]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [8]:
import konlpy 
from konlpy.tag import Okt 
okt = Okt()

In [9]:
# EDA 사용방법 (참고: https://github.com/toriving/KoEDA)
from koeda import EDA

eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.2, prob_rd=0.05
)

text = "아버지가 방에 들어가신다"

result = eda(text)
print(result)

아버지가 방에 들어가신다


In [10]:
# AEDA 사용방법
from koeda import AEDA

aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)

. 어머니가 , 집을 나가신다


sentence_1을 바꿀지, sentence_2를 바꿀지, sentence_1,2 모두 바꿀지? <br/>
5점 경우는 2700개의 데이터를 추가로 만들고 싶다 <br/>

<실험>

1. 문장은 사이클로 돌리고 sent 1,2,(1,2)는 random으로 선택하자
2. sentence 1,2 둘 다 aug

In [11]:
from koeda import AEDA
import pandas as pd
import numpy as np
import copy


def get_preprocessed_label(df):
    """_summary_

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    for i in range(len(df)):
        #df.loc[i, "preprocessed_label"] = round(df.loc[i, "label"])
        df.loc[i, "preprocessed_label"] = df.loc[i, "label"]

    return df


def concat_AEDA_sent(df_label, aug_nums):
    """라벨 x만 있는 데이터프레임에 aug_nums 만큼 AEDA된 데이터를 추가

    Args:
        df_label (DataFrame): 라벨 x만 있는 데이터프레임
        aug_nums (int): 증강하고자 하는 수
    Returns:
        AEDA가 추가된 라벨 x의 데이터프레임
    """
    np.random.seed(12)
    aug_idx = np.random.randint(0, 3, size=aug_nums)
    aug_idx = [2]*aug_nums
    dataset_idx = 0
    aug_df_label = copy.deepcopy(df_label)

    aeda = AEDA(
        morpheme_analyzer="Okt",
        punc_ratio=0.3,
        punctuations=[".", ",", "!", "?", ";", ":"],
    )

    for i in range(aug_nums):
        if dataset_idx >= len(df_label):
            dataset_idx = 0

        origin_id = df_label.iloc[dataset_idx]["id"]
        origin_source = df_label.iloc[dataset_idx]["source"]
        origin_sentence_1 = df_label.iloc[dataset_idx]["sentence_1"]
        origin_sentence_2 = df_label.iloc[dataset_idx]["sentence_2"]
        origin_label = df_label.iloc[dataset_idx]["label"]
        origin_binarylabel = df_label.iloc[dataset_idx]["binary-label"]
        origin_preprocessed_label = df_label.iloc[dataset_idx]["preprocessed_label"]

        if aug_idx[i] == 0:  # sent_1만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_1"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent],
                    "sentence_2": [origin_sentence_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        elif aug_idx[i] == 1:  # sent_2만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [origin_sentence_1],
                    "sentence_2": [aug_sent],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        else:  # sent_1과 2를 모두 aug
            sent_1 = aug_df_label.iloc[dataset_idx]["sentence_1"]
            sent_2 = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent_1 = aeda(sent_1)
            aug_sent_2 = aeda(sent_2)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent_1],
                    "sentence_2": [aug_sent_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        dataset_idx += 1

    return aug_df_label


def AEDA_data():

    train = pd.read_csv("train.csv")
    train = get_preprocessed_label(train)

    train_0 = train[train["preprocessed_label"] == 0].reset_index(drop=True)
    train_1 = train[train["preprocessed_label"] == 1].reset_index(drop=True)
    train_2 = train[train["preprocessed_label"] == 2].reset_index(drop=True)
    train_3 = train[train["preprocessed_label"] == 3].reset_index(drop=True)
    train_4 = train[train["preprocessed_label"] == 4].reset_index(drop=True)
    train_5 = train[train["preprocessed_label"] == 5].reset_index(drop=True)
    
    train_1_aug = concat_AEDA_sent(train_1, 1323)
    train_2_aug = concat_AEDA_sent(train_2, 1906)
    train_3_aug = concat_AEDA_sent(train_3, 1647)
    train_4_aug = concat_AEDA_sent(train_4, 936)
    train_5_aug = concat_AEDA_sent(train_5, 2750)

    train_0 =  pd.concat([train_0, train_1_aug, train_2_aug, train_3_aug, train_4_aug, train_5_aug], ignore_index=True)
    
    auged_df = train_0.sample(frac=1).reset_index(drop=True)
    
    return auged_df

In [12]:
final_df = AEDA_data()
final_df2 = final_df.copy()
final_df

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-4033,nsmc-sampled,도모토쯔요시 의외의 연기실력.........!,링 시리즈의 결정판...,0.0,0.0,0.0
1,boostcamp-sts-v1-train-2090,petition-sampled,", 가상화페 폐지 . 반대합니다.","가상 , 화폐 폐지 반대합니다.",5.0,1.0,5.0
2,boostcamp-sts-v1-train-9223,petition-rtt,법원은 동일한 잣대 ! 를 ? ...._,"법원 , 은 동일한 기준을 사용합니다 ...",3.0,1.0,3.0
3,boostcamp-sts-v1-train-7588,nsmc-rtt,예능보며 화나는건 이게 처음일뜻ㅋㅋㅋ,예능을 보면서 화를 내는 건 처음이다.,4.0,1.0,4.0
4,boostcamp-sts-v1-train-2914,slack-sampled,잘참고 하겠습니다.,잘 먹어야 득근 됩니다,0.0,0.0,0.0
...,...,...,...,...,...,...,...
12144,boostcamp-sts-v1-train-4113,slack-rtt,해피해피 : 한 연휴 보내세요~,행복하고 행복한 : 휴일 보내세요!,3.0,1.0,3.0
12145,boostcamp-sts-v1-train-2755,nsmc-sampled,: 지금 봐도 상당히 재밌는 . 영화,! 다시 봐도 특이하고 ? 재밌는 영화,3.0,1.0,3.0
12146,boostcamp-sts-v1-train-4061,petition-sampled,! 크림하우스 조사해 : 주세요,; 크림하우스매트 조사해주세요,4.0,1.0,4.0
12147,boostcamp-sts-v1-train-4944,nsmc-sampled,강동원 영상화보 잘보았읍니다,감명깊게 잘보았습니다.,2.0,0.0,2.0


In [13]:
final_df2.to_csv('./train_auged1.csv', index=False)

sentence 1과 2를 바꾼 행들도 추가하자 (train)

In [14]:
train = pd.read_csv("./train_auged1.csv")

In [15]:
train.columns

Index(['id', 'source', 'sentence_1', 'sentence_2', 'label', 'binary-label',
       'preprocessed_label'],
      dtype='object')

In [16]:
train_copy = train.copy()
train['sentence_1'] = train_copy['sentence_2']
train['sentence_2'] = train_copy['sentence_1']
train

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-4033,nsmc-sampled,링 시리즈의 결정판...,도모토쯔요시 의외의 연기실력.........!,0.0,0.0,0.0
1,boostcamp-sts-v1-train-2090,petition-sampled,"가상 , 화폐 폐지 반대합니다.",", 가상화페 폐지 . 반대합니다.",5.0,1.0,5.0
2,boostcamp-sts-v1-train-9223,petition-rtt,"법원 , 은 동일한 기준을 사용합니다 ...",법원은 동일한 잣대 ! 를 ? ...._,3.0,1.0,3.0
3,boostcamp-sts-v1-train-7588,nsmc-rtt,예능을 보면서 화를 내는 건 처음이다.,예능보며 화나는건 이게 처음일뜻ㅋㅋㅋ,4.0,1.0,4.0
4,boostcamp-sts-v1-train-2914,slack-sampled,잘 먹어야 득근 됩니다,잘참고 하겠습니다.,0.0,0.0,0.0
...,...,...,...,...,...,...,...
12144,boostcamp-sts-v1-train-4113,slack-rtt,행복하고 행복한 : 휴일 보내세요!,해피해피 : 한 연휴 보내세요~,3.0,1.0,3.0
12145,boostcamp-sts-v1-train-2755,nsmc-sampled,! 다시 봐도 특이하고 ? 재밌는 영화,: 지금 봐도 상당히 재밌는 . 영화,3.0,1.0,3.0
12146,boostcamp-sts-v1-train-4061,petition-sampled,; 크림하우스매트 조사해주세요,! 크림하우스 조사해 : 주세요,4.0,1.0,4.0
12147,boostcamp-sts-v1-train-4944,nsmc-sampled,감명깊게 잘보았습니다.,강동원 영상화보 잘보았읍니다,2.0,0.0,2.0


In [17]:
result = pd.concat([final_df, train])
result
result.to_csv('./train_auged2.csv', index=False)

In [18]:
result

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-4033,nsmc-sampled,도모토쯔요시 의외의 연기실력.........!,링 시리즈의 결정판...,0.0,0.0,0.0
1,boostcamp-sts-v1-train-2090,petition-sampled,", 가상화페 폐지 . 반대합니다.","가상 , 화폐 폐지 반대합니다.",5.0,1.0,5.0
2,boostcamp-sts-v1-train-9223,petition-rtt,법원은 동일한 잣대 ! 를 ? ...._,"법원 , 은 동일한 기준을 사용합니다 ...",3.0,1.0,3.0
3,boostcamp-sts-v1-train-7588,nsmc-rtt,예능보며 화나는건 이게 처음일뜻ㅋㅋㅋ,예능을 보면서 화를 내는 건 처음이다.,4.0,1.0,4.0
4,boostcamp-sts-v1-train-2914,slack-sampled,잘참고 하겠습니다.,잘 먹어야 득근 됩니다,0.0,0.0,0.0
...,...,...,...,...,...,...,...
12144,boostcamp-sts-v1-train-4113,slack-rtt,행복하고 행복한 : 휴일 보내세요!,해피해피 : 한 연휴 보내세요~,3.0,1.0,3.0
12145,boostcamp-sts-v1-train-2755,nsmc-sampled,! 다시 봐도 특이하고 ? 재밌는 영화,: 지금 봐도 상당히 재밌는 . 영화,3.0,1.0,3.0
12146,boostcamp-sts-v1-train-4061,petition-sampled,; 크림하우스매트 조사해주세요,! 크림하우스 조사해 : 주세요,4.0,1.0,4.0
12147,boostcamp-sts-v1-train-4944,nsmc-sampled,감명깊게 잘보았습니다.,강동원 영상화보 잘보았읍니다,2.0,0.0,2.0


다음 방법은 생성모델을 활용한 방법으로 Conditional BERT Contextual Augmentation 논문에 소개되었다. 

기존 BERT에서는 token embedding + segment embedding + positional embedding 으로 representation을 구성하지만, 

conditional BERT의 경우 token embedding + label embedding + positional embedding 으로 representation을 구성하고, 
label을 부착한 상태로 데이터셋을 MLM task로 pretraining한다. 
이후에 mask token을 replace하는 것과 마찬가지로 label에 대하여 token replace를 수행한다.